# Speech latent feature discovery for multi-label forensic profiling

Project for IWBF 2018

----

**Objective:** Find latent representations in speech that can predict forensic aspects of human.


## Update ~12/11/2017

1. Write-up:

    https://www.overleaf.com/12702004wrrmdybmpqsq#/48448185/

2. Implementation
    1. Data: TIMIT, Alcohol, (SRE)
    2. Model
        ![model](./write-up/figs/model.png)  
    3. Current attributes: speaker-id, gender, dialect, age, height, drunk
    4. [Code](./src)
    
3. Final poster for convex

## Update ~12/18/2017

1. Experiments
    1. Currently using latent codes from autoencoder to do multi-label classification and regression.
    2. It works well on certain tasks (reconstruction, gender, height), but not well on other tasks (id, dialect, age).
    3. Including more datasets (SRE, Yandong's face data)

large model openset (100 epochs, batch 4)

|Label | Reconstruction Loss ($l_2^2$) | ID Acc (%) | Gender Acc (%) | Dialect Acc (%) | Age MSE | Height MSE |
|:----|----:|---:|---:|---:|----:|---:|
| Train | 0.1417 | 75.15 | 97.15 | 80.40 | 6.6931 | 0.0668 |
| Test | 0.1847 | 3.59 | 92.83 | 14.73 | 81.1115 | 0.0818 |
 
2. Writing paper

3. Todo next week
    1. Finish draft of the paper.
    2. Improve the model.
    

## Update ~12/19/2017

1. Experiments
    1. Observations: ID, Age and Dialect seem to be correlated (similar trends of loss), whereas Gender and Height seem to be independent (not speaker-dependent).
    2. Back to closed set of speakers -- openset is far more difficult.
    3. Reduce model size to speed up training.

## Update ~12/20/2017

1. Experiments
    1. Train on size-reduced model: significantly reduced model size, less layers, less parameters, smaller z dimension.
    2. Use closed set speakers, split (0.8 / 0.2).
    3. Observations: 
    
small model openset (300 epochs, batch 4)

|Label | Reconstruction Loss ($l_2^2$) | ID Err (%) | Gender Err (%) | Dialect Err (%) | Age MSE | Height MSE |
|:----|----:|---:|---:|---:|----:|---:|
| Train | 0.1168 | 0.98 | 0.02 | 1.45 | 1.1543 | 0.0980 |
| Test | 0.1529 | 98.60 | 3.81 |  82.36 | 83.5361 | 0.0981 |

## Update ~12/20/2017

1. Experiments
    1. Observations: while the train losses monotonically decreases, the test losses for id and dialect strangely increases.

![strange-loss](./write-up/figs/strange_loss.png)

small model closedset (300 epochs, batch 16)

|Label | Reconstruction Loss ($l_2^2$) | ID Err (%) | Gender Err (%) | Dialect Err (%) | Age MSE | Height MSE |
|:----|----:|---:|---:|---:|----:|---:|
| Train | 0.1209 | 0.15 | 0.00 | 0.31 | 0.5237 | 0.0177 |
| Test | 0.1609 | 87.54 | 0.0150 | 77.97 | 60.2400 | 0.0773 |

## Update ~12/28/2017

Did not do much this week ...

1. Found the stupid bug in code, now all losses decreasing.
    
small model closedset (less speakers)

|Label | Reconstruction Loss ($l_2^2$) | ID Err (%) | Gender Err (%) | Dialect Err (%) | Age MSE | Height MSE |
|:----|----:|---:|---:|---:|----:|---:|
| Train |  | 32.40 |  |  |  |  |
| Test |  | 48.33 | | |  ||

2. Paper

    https://www.overleaf.com/12702004wrrmdybmpqsq#/48448185/

3. Todo

    1. Extend to multi-datasets
    2. Have to finish the paper !!!


## Update ~12/30/2017

1. Multitask learning (MTL)
    1. Models
        1. Neural models
            1. Hard parameter sharing
            2. Soft parameter sharing
            3. **what to share in model?**

                ** sharing information with unrelated tasks might impede learning individual tasks**
        2. Non-neural models [assume model parameter $W$ of size ($d$-feature dims $\times$ $T$-tasks)]
            1. sparsity across tasks through norm regularization
                1. block-sparse regularization: mixed $l_1 / l_q$ norms
                2. group lasso
                3. trace norm regularize: low rank
                4. combine block-sparse and element-wise sparse
            2. modelling the relationships between tasks
                1. clustering constraints: task mean and between-task variance; possibly add within-task variance
                2. more complex structures: graph, tree, k-nn
                3. Bayesian
                    1. Gaussian process with shared covariance
                    2. mean task-dependent and a clustering of the tasks using a mixture distribution
                    3. Dirichlet process and enable the model to learn the similarity between tasks as well as the number of clusters
                    4. hierarchical Bayesian model -- a latent task hierarchy
                    5. actual tasks are linear combination of small number of latent basis tasks
        3. Auxiliary tasks
            1. related task
            2. adversarial task
            3. hints: predict features
            4. attention
            5. quantization smoothing
            6. representation learning
            7. **what auxiliary tasks are helpful?**
            
            e.g.  two tasks are $\mathcal{F}$-related if the data for both tasks can be generated from a fixed probability distribution using a set of transformations $\mathcal{F}$
            
            Task similarity is not binary, but resides on a spectrum. More similar tasks should help more in MTL, while less similar tasks should help less. Allowing our models to learn what to share with each task might allow us to temporarily circumvent the lack of theory and make better use even of only loosely related tasks. However, we also need to develop a more principled notion of task similarity with regard to multi-task learning in order to know which tasks we should prefer.
            
             **Existing problems**: task similarity, relationship, hierarchy, and benefit for MTL

2. **Multitask learning and Adversarial learning**
    1. Domain adaptation ?
    2. MTL with loosely related tasks: multiple adversarial learning?


## Update ~01/01/2018

1. Address the problems analyzed in **Update ~12/30/2017**
    1. Need a sharing mechanism among tasks
    2. Need a latent domain adaptation mechanism
    3. Weighted loss w.r.t. the task uncertainty

## Update ~01/02/2018

1. Experiments
    0. **Two difficulties**
        1. Model was not able to handle single but difficult task (e.g., id, age), neither multiple tasks. --> need better model
        2. The tasks are loosely relevant. Some tasks bias other tasks. --> need proper model sharing, loss weighting, auxillary tasks
    1. Tried different sharing mechanisms but not work
        1. sharing part of encoder but with different latent codes for different tasks
        2. sharing latent codes (including varying latent code distributions, e.g. Gaussians with centered mean but different variances)
        3. sharing part of decoder
    2. Changed model to Resnet-like structure
        1. combine multi-resolution features
    3. Applying weighting to different task losses
        1. hard-assigned weights: assign larger weights to tough tasks.
        2. soft-assigned weights: probabilistically reformulate the task objectives (for regression, model likelihood as a Gaussian; for classification, use softmax), and then use objective variance / uncertainty to adjust weights.
    4. Trying multi-stage autoencoders
    5. Attention model?

## Update ~01/03/2018

1. Experiments
    1. Resnet-like structure seems to improve accuracy than standard CNN.
    2. Use task-wise learning rates and task-wise early stopping.
    3. Use regularization on shared latent representation 

## Update ~01/04/2018

0. Conclusions
    1. Based on previous observations and discussion, we need
        0. Model: Resnet, probably add attention
        1. Model sharing: must share from latent level
        2. Related tasks: design auxillary tasks, corresponding loss and learning strategy
        3. Regularization on latent space: currently tried sparse, $l_2$, and normal
1. Experiments
    1. Resnet-like structure generally improve test performance (but not much).
    
    [See some loss curves](./write-up/figs/loss_records)
    
    2. Running task-wise early stopping with smaller learning rate.
    
    [Ref: Facial Landmark Detection by Deep Multi-task Learning](https://link.springer.com/chapter/10.1007/978-3-319-10599-4_7)
    
    [Ref: PhD Thesis: Multitask learning](http://reports-archive.adm.cs.cmu.edu/anon/1997/CMU-CS-97-203.pdf)
    ![Example of MTL on nine 1D-ALVINN tasks](./write-up/figs/eg_mtl_loss_curves.png)
    
    3. Running single task.
    4. Using ID as main task, and others as auxillary tasks.
    
    Other tasks can be viewed as sub-tasks of ID.
    
    5. Redesign loss
        
        For regression: the likelihood $p(y_1 | f(x)) = \mathcal{N}(f(x), \sigma_1^2)$
        
        For classification: $p(y_2 | f(x)) = \text{softmax}(\frac{1}{\sigma_2^2}f(x))$
        
        The total log-likelihood for both regression and classification tasks: $\mathcal{L}(\sigma_1, \sigma_2) \approx \frac{1}{2\sigma_1^2}\|y_1 - f(x)\|_2^2 - \frac{1}{2\sigma_2^2}\log\text{softmax}(y_2, f(x)) + \log\sigma_1^2 + \log\sigma_2^2$
        
        Hence the variance controlls the contribution of each task.
        
        Can further extend to generalized linear mixed model.

## Update ~01/07/2018

1. Experiments
    1. Apply task-wise early stopping
    
        Loss reduced for ID, Dialect and Age.
    2. Use wasserstein loss for Autoencoder
        
        Loss reduced for reconstruction. 
    3. Learning strategy: train AE more before training Discriminators
    
        Improved performance.
    4. Conjecture: two adversaries in learning
        1. Adversary between Reconstruction and Discrimination
        
            Consider X -f- z -g- X' and X -f- z -h- y: $(g \circ f)(X) = X'$, $(h \circ f)(X) = y$, then $f = g^{-1} \circ I$ and $(h \circ (g^{-1} \circ I))(X) = y$. Hence the discrimination process invloves in inverting reconstruction process.
            
        2. Adversary among discriminators
        
            Each decoding process has true, underlying $\{\mathcal{Z}\}$, but the encoded $\{\mathcal{\widetilde{Z}}\}$ differ with $\{\mathcal{Z}\}$. There is a game among learning these latent representations.
    
    5. Todo: Apply probabilistic early stopping

## Update ~01/08/2018

1. Experiments
    1. Dynamic task-wise stopping and resuming
        
        Use learning curve statistics to regularize learning: at step $t$, stop task $\alpha$ if 
        
        $$\frac{\text{med}_kE_{\text{train}}^{\alpha}}{\left|\text{mean}_kE_{\text{train}}^{\alpha} - \text{med}_kE_{\text{train}}^{\alpha}\right|} \cdot \frac{\left|E_{\text{test}}^{\alpha}(t) - \min_tE_{\text{train}}\right|}{\lambda^{\alpha}\min_tE_{\text{train}}} > \text{threshold}$$,
        
        and resume if smaller than the threshold, where $\text{med}_kE_{\text{train}}^{\alpha}$ is the median of the recent $k$ train errors for task $\alpha$, $E_{\text{test}}^{\alpha}(t)$ is the test error at step $t$, $\min_tE_{\text{train}}$ is the minimum of the total $t$ train errors, and $\lambda^{\alpha}$ is the loss weight for task $\alpha$.
        
    2. Loss
        1. Use dual of wasserstein distance for reconstruction; mean absolute deviation for continuous attributes prediction --> weak convergence
        2. Todo: use probabilistic formulation for all tasks and derive weaker metric;
        
            also use it to adjust dynamic task-wise stopping and resuming.
            
            

## Update ~01/10/2018

0. Conclusions
    1. Different task has different learning pace, and aids main task in different phase.
    2. Wake and Sleep phase existed in learning.
        
        The following figures show during the learning, there exists **wake phase (level 0) and sleep phase (level 1)** for different tasks.
        
        All figures are from Resnet-like structure on TIMIT data, using Wasserstein distance for reconstruction, mean absolute deviation for age and height, with **different dynamic stopping and resuming thresholds**.
        
        The **Purpule and Red** curves represent currently best performance.
        
        Wake and sleep for age and dialect. 
        ![age_dia_stop](./write-up/figs/loss_records/age_dia_stop.png)
        
        Wake and sleep for gender and height. 
        ![gen_hgt_stop](./write-up/figs/loss_records/gen_hgt_stop.png)
        
        Wake and sleep for id. 
        ![id_stop](./write-up/figs/loss_records/id_stop.png)
    
1. Experiments
    1. Dynamic stopping and resuming significantly reduces multi-task test errors.
    
300 epochs

|Label | Reconstruction Loss ($Wass_1$) | ID Err (%) | Gender Err (%) | Dialect Err (%) | Age MAE (yr) | Height MAE (cm) |
|:----|----:|---:|---:|---:|----:|---:|
| Train | 0.2210 | 28.87 | 1.76 | 47.27 | 5.298 | 0.2647 |
| Test | 0.2508 | 39.76 | 1.03 | 68.60 | 5.405 | 0.2724 |

[See some loss curves (train_err_id_age_dynamic_stop ...) (Red and Purple curves are latest.)](./write-up/figs/loss_records)
    
    2. Currently using task variance to regularize wake and sleep, task weights, and task learning rates.
    
    Results pending.
    
    3. More auxiliary tasks may help.

## Update ~01/16/2018

1. Experiments
    1. Task variance prediction & use variance to adjust dynamic stopping as well as learning rates.
        
        $\Sigma_t = (h_{\theta_t}^*)^2 - (h_{\theta_t}^*h_{\theta_t})^2[h_{\theta_t}^2 + \sigma_t^2]^{-1}$

## Update ~01/19/2018

1. Experiments
    1. Use task variance to control task loss weights and dynamic stopping -> reduced age and height loss, seems work for regression.
    2. Test hypothesis: for a rival task pair (A, B), the loss for task A stops descreasing because task B hinges task A; stopping task B helps the loss of task A to resume decreasing. For a friends pair (A, B), the loss for task A decreases further if task B joins A, and vice versa.

## Update ~01/22/2018

1. Experiments
    1. Continue testing hypothesis on friend and rival pairs.

## Update ~01/24/2018

1. Experiments
    1. Finished a thorough series of tests:
        1. H1: Friend task pair (A, B) helps each other to reduce loss, namely, if A has a high variance, joining B reduces loss(A), and vice versa.
        
        Friend pair shares similar optimization paths, joining a friend task adjusts update steps and provides larger gradients. 
        
        Test 1: (Hgt, Gen), (Hgt, Age), (Hgt, Dia)
        
        2. H2: Rival task pair (A, B) pulls each other off from optimal path to high error path; loss for stronger one reduces, while loss for weaker one increases.
        
        Rival pair has diverse optimization path, joining a rival leads to competing update directions.
        
        Test 2: (Id, Hgt), (Id, Dia), (Id, Age)
        
        3. H2-2: For a rival pair (A, B), stopping one helps the loss of the other to drop, and vice versa.
        
        Test 2-2: (Id, Hgt), (Id, Dia), (Id, Age)
        
    2. Performance improved.
    
2. Theory
    1. Derive a manifold view of the multi-task learning scheme (can also adopted to adversarial learning scheme).
    
    ![manifold-view](./write-up/figs/manifold_view.png)

## Update ~01/28/2018

1. Conclusions from experiments
    1. Although we do not yet have a way describing / quantitizing the relation and hierarchy among the tasks, the pair-wise experiments show that the tasks have a *relative closeness*: 

    ![task-relation](./write-up/figs/task_relation.png)
    
    We do not want to model the *relative closeness* since it's dynamic, complicated and task-dependent. Rather, we seek for an alternative that is task-independent.
    
    2. Dynamic stopping does not help much in these pair-wise experiments.
    
2. Theories
    
    We develop a link between the *curves on the latent manifold* and the *loss maps*; we want to characterize the relation between the *direction of these curves* (*tangent bundles*) and the *loss trends*.
    1. Describe the joint loss manifold
    2. Describe the maps among manifolds
    3. Describe the maps between the curves on latent manifold and the loss manifold
    4. Characterize the tangent bundle


## Update ~01/30/2018

1. Hypos, experiments and conclusions
    1. H1: For a fixed latent manifold $\mathcal{Z}$, each task in the multi-dicrimination process is independent of the others, follows its own optimization path, and reaches to its own optimal.
    2. T1: Train an optimal AE first, fix Z, then train on multi-tasks. Test on single task, two-task pairs, and all tasks.
    3. C1: H1 negative. Even for fixed Z, the tasks intertwine with each other. Ex., for err_id, id+age < all; for err_age, all < age+id. In addition, compared to simultaneously train AE and Ds, fixing AE first and then training Ds generally leads to slower learning of dia, id, hgt, but faster on age, gen. This shows that dia, id, and hgt are indeed hard to learn, while gen and age are relatively easy.

## Update ~02/04/2018

1. Modeling dynamics in multitask learning

    ![manifold_model](./write-up/figs/manifold_model.png)


- The same theory can be applied to adversarial learning using **Morse flow**.

## Update ~02/26/2018

1. Paper

    https://www.overleaf.com/13976332njvxtpgzcsnm#/54189324/
    
    First thoughts on identifying age-related dimensions
    1. train AE model against age first, and mask latent dimensions
    2. use other tasks to identify confounding dimensions
    
2. Experiment
    1. Preparing data and baselines

## Update ~03/05/2018

1. IS2018
    1. Baselines
        1. GMM supervector + [PCA / LDA] + [WCCN] + SVR
        2. GMM supervector -> IVector + [PCA / LDA] + [WCCN] + SVR
        3. AE + Regression
    2. Improvements
        1. Regression to classification
    3. Paper
        1. https://www.overleaf.com/13976332njvxtpgzcsnm#/54189324/

## Update ~03/12/2018

1. IS2018
    1. Experiments
        1. Implement multi-level classification with soft thresholding
            1. Data: 1-min segments
            0. Feat: ivector
            1. Classifier: SVM with linear kernel, separate male / female
            2. Partition penalty: no
            3. For 1-level with (young, old), it got the middle value (35)
            4. For multi-level, it broke
            
        2. Implement multi-level classification with grid search
            1. Just implemented

## Update ~03/18/2018


__IS2018 Experiments__

1. Data: 

    1. ivec400: ubm512 from 20 spks, enroll 9346 spks, with EER = 29.438%

    2. ivec400_large: ubm512 from 3500 spks, enroll 9346 spks, tot 12846 spks (with some overlap between ubm and enroll, have actually 11971 spks), with EER = 
    
    2. ivec400_large: ubm512 from 200 spks, enroll 9346 spks, with EER = 11.242%

    3. ivec200: ivec400 + lda / pca

2. Mean: 36.7592, Median: 35

| data | svr | result |
|------|-----|--------|
| Shahan's partition, non-overlap, openset, tot 11913 spks, divide to male / female, split to train:dev:test = [4813f/3100m]:1000:1000 | kernel = linear, C = 1.0, epsilon = 0.2 | train mae_f: 9.4632, test mae_f: 12.8851, train mae_m: (), test mae_m: () |

3. SVR

            
| data | svr | result |
|------|-----|--------|
| dev 9346 spks, tot 93487 features, split to train:test = 0.2:0.8 | kernel = rbf, C = 1.0, epsilon = 0.2 | train mae: 9.5698, test mae: 10.9259 |
| dev 9346 spks, tot 93487 features, split to train:test = 0.2:0.8 | kernel = linear, C = 1.0, epsilon = 0.2 | train mae: 10.1750, test mae: 10.9117 |
| dev 9346 spks, tot 93487 features, divide to male / female, each split to train:test = 0.2:0.8 | kernel = linear, C = 1.0, epsilon = 0.2 | train mae_f: (), test mae_f: (), train mae_m: (), test mae_f: () |
| Shahan's partition, non-overlap, openset, tot 11913 spks, divide to male / female, split to train:dev:test = [4813f/3100m]:1000:1000 | kernel = linear, C = 1.0, epsilon = 0.2 | train mae_f: 9.1426, test mae_f: 12.9088, train mae_m: (), test mae_m: () |

4. l-level SVM, smooth

Linear SVM, hinge loss, slack, C = 1.0

| data | loss | class err | threshold | + triviality loss |
|------|------|-----------|-----------|------------|
| train_f | 36767.9570 | .0063 | 71.3 | 37.5 |
| dev_f | 7936.5679 | 0.0294 | 85.2 | 55.5 |
| test_f | | | | |
| f: train + dev -> test | | | | |
| train_m | | | | |
| dev_m | | | | |
| test_m | | | | |
| m: train + dev -> test | | | | |
| train | | | | |
| dev | | | | |
| test | | | | |
| train + dev -> test | | | | |

5. 1-level logistic regression, smooth

| data | loss | class err | threshold | triviality |
|------|------|-----------|-----------|------------|
| train_f | 0.2090 | 0.0038 | 35.6 | |
| dev_f | 0.0171 | 0.0181 | 33.2 | |
| test_f | | | | |
| f: train + dev -> test | | | | |
| train_m | | | | |
| dev_m | | | | |
| test_m | | | | |
| m: train + dev -> test | | | | |
| train | | | | |
| dev | | | | |
| test | | | | |
| train + dev -> test | | | | |

6. multi-level SVM, smooth

| data | loss | class err | threshold | triviality |
|------|------|-----------|-----------|------------|
| train_f |  |  |  | |
| dev_f | | | | |
| test_f | | | | |
| f: train + dev -> test | | | | |
| train_m | | | | |
| dev_m | | | | |
| test_m | | | | |
| m: train + dev -> test | | | | |
| train | | | | |
| dev | | | | |
| test | | | | |
| train + dev -> test | | | | |

7. multi-level logistic regression, smooth

| data | loss | class err | threshold | triviality |
|------|------|-----------|-----------|------------|
| train_f | | | | |
| dev_f | | | | |
| test_f | | | | |
| f: train + dev -> test | | | | |
| train_m | | | | |
| dev_m | | | | |
| test_m | | | | |
| m: train + dev -> test | | | | |
| train | | | | |
| dev | | | | |
| test | | | | |
| train + dev -> test | | | | |

9. Evaluation: SVR evaluated on age partitions from our model

8. classification using ad-hoc boundaries

    7 groups of ages

9. regression from our method (as going down the levels)
   
   convert classification output to regression

## Update ~03/26/2018

1. What's done this week
    1. Paper draft w/ partial results
    
2. To do next week
    1. Complete results
        1. Perform PLDA on ivectors
        2. Cross-compare regression to classification and classification to regression
    2. Joint optimization on tree


## Update ~04/02/2018

1. Age estimation via model tree classification
    1. Stop the branch of the tree to grow if further partition leads to 1-age-only group / highly unbalanced subsets.
    2. Tried different training configurations
        1. num of epochs per level
        2. LDA dims per level
        3. If doing WCCN per level
        4. Learning rate per level
        5. C in SVM per level
    2. Comparing ivector regression to model tree classification
    
    For each test, put the predicted age from regression into the interval decided by the partition from model tree approach, and then compute the classification error.
    
Female, test

| Level | Threshold | Model Tree Err | Regression-to-Classification Err | Avg MT / Reg|
|:-------|--------:|------:|--------------------------------------------:|---:|
| 1 | 34.38 | 0.22 | 0.25 | 0.22 / 0.25|
| 2 | 20.07, 76.77 | 0.28, 0.35 | 0.30, 0.31 | 0.315 / 0.305 |
| 3 | 17.00,  30.44,  37.90,  81.87 | 0.10,  0.35,  0.24,  0.45 | 0.22,  0.30,  0.27,  0.37 | 0.285 / 0.29 |
| 4 | 15.00, 18.00, 24.40, 31.97, 35.32, 52.70, 80.05, 81.83 | 0.01, 0.11, 0.45, 0.40, 0.44, 0.40, 0.87, 0.40 | 0.21, 0.37, 0.40, 0.25, 0.27, 0.48, 0.37, 0.30 | 0.385 / 0.331|

Male, test

| Level | Threshold | Model Tree Err | Regression-to-Classification Err | Avg MT / Reg|
|:-------|--------:|------:|--------------------------------------------:|---:|
| 1 | 31.09 | 0.23 | 0.24 | 0.23 / 0.24|
| 2 | 23.87, 76.68 | 0.33, 0.36 | 0.34, 0.32 | 0.345 / 0.34 |
| 3 | 19.48,  28.43,  42.18,  82.61 | 0.47,  0.35,  0.39,  0.39 | 0.34,  0.45,  0.44,  0.37 | 0.4 / 0.4 |
| 4 | 16.03, 20.15, 25.15, 33.17, 37.45, 55.00, 79.45, 82.61 | 0.41, 0.47, 0.37, 0.45, 0.36, 0.36, 0.30, 0.24 | 0.48, 0.31, 0.23, 0.46, 0.45, 0.39, 0.37, 0.49 | 0.37 / 0.40|

**Observations**

1. Errs went higher with levels

2. Model tree got high err (>50%) at the 5-th level

3. Model tree is on a par with regression

4. The tree went unbalanced at the bottom levels, weighting more on the right side / elder side

## Update ~04/09/2018

1. Age classification with soft boundaries for dvectors
    0. The model tree can now go into arbitrary depth
    1. Got regression-to-classification results for dvectors
    2. Got classification-to-classification results for ivectors and dvectors
    3. Integrated feature extraction into the model tree framework
    
**Regression-to-classification**
![](./write-up/figs/model_tree_result_female.png)
![](./write-up/figs/model_tree_result_male.png)

**Classification-to-regression**

$\mathbb{E}(age\mid X) = \sum_{p}\mathbb{E}(age\mid part_p)p(part_p\mid X)$

where $p(part_p\mid X) = \prod_{parent, child \in \mathcal{V}_k, \mathcal{V}_k \in \mathcal{V}_{tree}} p(child \mid parent)$

Female, test

| Feat | SVR MAE | MT MAE |
|:-----|--------:|-------:|
| Ivec | 8.7531 | 9.1086 |
| Dvec | 8.0244 | 7.7364 |

Male, test

| Feat | SVR MAE | MT MAE |
|:-----|--------:|-------:|
| Ivec | 9.2162 | 10.2564|
| Dvec | 8.2979 | 8.1004 |

**Integrating feature extraction into model tree**

1. Added a MLP as feature extractor, and extended the model parameters from $(t, w_{SVM})$ to $(t, w_{SVM}, w_{MLP})$.

2. Optimize over entire tree
    
    The loss for regression is
    
    $L_{reg} = \mathbb{E}_{\mid X}\left | trueage - \sum_{p}\mathbb{E}(age\mid part_p)p(part_p\mid X) \right |$
    
    where $p(part_p\mid X) = \prod_{parent, child \in \mathcal{V}_k, \mathcal{V}_k \in \mathcal{V}_{tree}} p(child \mid parent)$
    
    $p(child \mid parent) \longleftarrow SVM(parent)$, for $k = 1, ..., L$
    
    for the first level ($k=0$), $p(child \mid X) \longleftarrow MLP(X)$
    
    The loss for classification in level $k$ is
    
    $L_{clas}^k = \sum_{p}p\left(x_p^{k} \mid pa(x_p^{k})\right)\left[Err\left(y(x_p^k), \widehat{y}(x_p^k, t_p^k)\right) + \mathcal{T}(\mathcal{\pi}(x_p^k))\right]$
    
    The total loss for the tree is
    
    $\mathcal{L}_{tree} = L_{reg} + \sum_{k=1}^{L}L_{clas}^k$

## Update 04/16/2018

1. Improving classification-to-regression MAE

    MAE on dev, dvector

    $\lambda \mathcal{T} + (1-\lambda) Err_{class}$

| $\lambda$ | Female | Male |
|:----------:|------:|------:|
| 0.1 | 14.82 | 10.66 |
| 0.2 | 11.72 | 10.83|
| 0.3 | 10.21 | 10.85 |
| 0.4 | 9.10 | 10.34 |
| 0.5 | 12.83 | 14.00 |
| 0.6 | **7.47** | 12.47|
| 0.7 | 13.52 |**7.86** |
| 0.8 | 14.52 |12.91 |
| 0.9 | 13.14 | 12.35|

2. Implemented model tree with feature extraction

3. Implementing optimization over tree

4. Paper

    1. Yap, T. F., Epps, J., Choi, E. H., & Ambikairajah, E. (2010, March). Glottal features for speech-based cognitive load classification. In Acoustics Speech and Signal Processing (ICASSP), 2010 IEEE International Conference on (pp. 5234-5237). IEEE.

    [Summary Slides (Online)](https://drive.google.com/open?id=1OaqZofHh6XlwtIR3SFdGv0R0eAtqOXeZ)

    [Summary Slides (Local)](./write-up/GlottalFeaturesForSpeech-basedCognitiveLoadClassification.pptx)

    2. Airas, M. (2008). TKK Aparat: An environment for voice inverse filtering and parameterization. Logopedics Phoniatrics Vocology, 33(1), 49-64.
    
    3. Alku, P. (1992). Glottal wave analysis with pitch synchronous iterative adaptive inverse filtering. Speech communication, 11(2-3), 109-118.
    
    4. Naylor, P. A., Kounoudes, A., Gudnason, J., & Brookes, M. (2007). Estimation of glottal closure instants in voiced speech using the DYPSA algorithm. IEEE Transactions on Audio, Speech, and Language Processing, 15(1), 34-43.

## Update 04/23/2018
1. Drafting paper

https://www.overleaf.com/15602840jwctmypqkhrd#/59268244/

2. Neural model tree
    0. No improvement when brutal forcely stack a mlp: MAE 15.2953 for test female and dvector
    1. Dealing with variable tree structure

2. Extracting glottal features using Aparat toolbox

![](./write-up/figs/aparat_2.png)
![](./write-up/figs/aparat_1.png)
![](./write-up/figs/aparat_3.png)
![](./write-up/figs/aparat_4.png)
![](./write-up/figs/aparat_5.png)

3. Summer intern interviews

## Update 04/16/2018

1. Neural model tree

    1. Improving tree results: with coordinate descent and alternative triviality loss
        0. Coordinate descent: at each iteration, update the net parameter $\psi$, and then update the tree parameter $\mathbf{\Theta} := \bigcup\limits_{k=1}^{L}\{(t_k, \theta_k)\}$ with another coordinate descent
        1. Alternative triviality loss to avoid gradient vanishing: $\mathcal{T}(t, \mathbb{Y}) = (t - \text{med}(\mathbb{Y}))^2$
        2. Improved MAE -- female, dvector, test: 9.2192
        
    2. Variable tree structure
        1. Dynamic pruning: for each level, each iteration and $t_k$, if yielding trivial partition, prune the branch.
    
    3. Paper draft ++
        
        https://www.overleaf.com/15602840jwctmypqkhrd#/59268244/
        